In [1]:
import scanpy as sc
import numpy as np
import scgen as scg
import pandas as pd
import sys
sys.path.append("/work/users/username/projects/cov/integration/analysis")
import helper_VAE as hVAE
import scipy.sparse
import scgen
import matplotlib.pyplot as plt
import seaborn as sns
import harmonypy as hpy
import anndata
import random

import os

/home/sc.uni-leipzig.de/username/miniconda3/envs/scgen/lib/python3.9/site-packages/statsmodels/compat/pandas.py:65: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import Int64Index as NumericIndex


In [2]:
species = 'ma'
celltype = 'Neutrophils'

In [3]:
model_save_string = os.getcwd()
model_save_string = model_save_string + '/' + celltype + '/H0_01_model_' + celltype + '.pt'

In [4]:
model = scgen.SCGEN.load(model_save_string)

INFO     Using data from adata.X                                                             
INFO     Computing library size prior per batch                                              
INFO     Registered keys:['X', 'batch_indices', 'local_l_mean', 'local_l_var', 'labels']     
INFO     Successfully registered anndata object containing 4616 cells, 1531 vars, 5 batches, 
         1 labels, and 0 proteins. Also registered 0 extra categorical covariates and 0 extra
         continuous covariates.                                                              


/home/sc.uni-leipzig.de/username/miniconda3/envs/scgen/lib/python3.9/site-packages/scvi/data/_anndata.py:301: UserWarning: Training will be faster when sparse matrix is formatted as CSR. It is safe to cast before model initialization.
  warnings.warn(


In [6]:
adata = model.adata
lat_adata = hVAE.get_latent_representation_object(model,adata)

#hVAE.prepare_umap(lat_adata)

#sc.pl.umap(lat_adata,color = 'timepoint')

d0 = hVAE.filter_adata_obs(lat_adata,col_name='timepoint',val='d0')
d2 = hVAE.filter_adata_obs(lat_adata,col_name='timepoint',val='d2')
d3 = hVAE.filter_adata_obs(lat_adata,col_name='timepoint',val='d3')
d5 = hVAE.filter_adata_obs(lat_adata,col_name='timepoint',val='d5')
e14 = hVAE.filter_adata_obs(lat_adata,col_name='timepoint',val='e14')

lat_adata = hVAE.merge_adata(hVAE.merge_adata(hVAE.merge_adata(hVAE.merge_adata(e14,d2),d5),d3),d0)

delta = hVAE.get_delta_in_latent_space(d0,d2)

/home/sc.uni-leipzig.de/username/miniconda3/envs/scgen/lib/python3.9/site-packages/anndata/_core/anndata.py:1755: FutureWarning: The AnnData.concatenate method is deprecated in favour of the anndata.concat function. Please use anndata.concat instead.

See the tutorial for concat at: https://anndata.readthedocs.io/en/latest/concatenation.html
  warnings.warn(
/home/sc.uni-leipzig.de/username/miniconda3/envs/scgen/lib/python3.9/site-packages/anndata/_core/anndata.py:1755: FutureWarning: The AnnData.concatenate method is deprecated in favour of the anndata.concat function. Please use anndata.concat instead.

See the tutorial for concat at: https://anndata.readthedocs.io/en/latest/concatenation.html
  warnings.warn(
/home/sc.uni-leipzig.de/username/miniconda3/envs/scgen/lib/python3.9/site-packages/anndata/_core/anndata.py:1755: FutureWarning: The AnnData.concatenate method is deprecated in favour of the anndata.concat function. Please use anndata.concat instead.

See the tutorial for conca

In [7]:
test_set = sc.read_h5ad(celltype + '/H0_01_test_Neutrophils.h5ad')

lat_adata_test = hVAE.get_latent_representation_object(model,test_set)

lat_d0_test = lat_adata_test[lat_adata_test.obs['timepoint'] == 'd0']

lat_d0_test_shifted = hVAE.shift_adata_in_latent_space(lat_d0_test,delta)

lat_d0_test_shifted.obs['timepoint'] = 'd2 predicted'

lat_test_predicted = hVAE.merge_adata(lat_adata_test,lat_d0_test_shifted)

INFO     Input adata not setup with scvi. attempting to transfer anndata setup               
INFO     Using data from adata.X                                                             
INFO     Computing library size prior per batch                                              
INFO     Registered keys:['X', 'batch_indices', 'local_l_mean', 'local_l_var', 'labels']     
INFO     Successfully registered anndata object containing 2871 cells, 1531 vars, 5 batches, 
         1 labels, and 0 proteins. Also registered 0 extra categorical covariates and 0 extra
         continuous covariates.                                                              


/home/sc.uni-leipzig.de/username/miniconda3/envs/scgen/lib/python3.9/site-packages/scvi/data/_anndata.py:301: UserWarning: Training will be faster when sparse matrix is formatted as CSR. It is safe to cast before model initialization.
  warnings.warn(
/home/sc.uni-leipzig.de/username/miniconda3/envs/scgen/lib/python3.9/site-packages/anndata/_core/anndata.py:1755: FutureWarning: The AnnData.concatenate method is deprecated in favour of the anndata.concat function. Please use anndata.concat instead.

See the tutorial for concat at: https://anndata.readthedocs.io/en/latest/concatenation.html
  warnings.warn(


In [8]:
decoded_test = hVAE.decode_latent_object(model,lat_test_predicted,test_set)

In [11]:
decoded_test.obs['timepoint'].value_counts()

d2              1372
d5               683
d3               325
e14              246
d0               245
d2 predicted     245
Name: timepoint, dtype: int64

In [12]:
gene_expression_df = pd.DataFrame(decoded_test.X,columns = decoded_test.var.index,index = decoded_test.obs.index)

In [16]:
obs_df = decoded_test.obs[['timepoint','uniform_name_acurate_N']]

obs_df.to_csv(celltype + '/HO_03_meta_' + celltype + '_' + species + '.csv')

gene_expression_df.to_csv(celltype + '/HO_03_gene_expression_' + celltype + '_' + species + '.csv')